In [2]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [3]:
fname_url="https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip"
fname=fname_url.split("/")[-1]

In [4]:
!wget $fname_url


--2021-12-05 19:03:57--  https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip [following]
--2021-12-05 19:03:58--  https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16979362 (16M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  16.19M  88.8MB/s    in 0.2s    

2021-12-05 19:03:58 (88.8 MB/s) - ‘train.z

In [5]:
!mkdir "train"
!rm ./train/Train.csv

rm: cannot remove './train/Train.csv': No such file or directory


In [6]:
!unzip $fname -d "./train"
!rm train.zip

Archive:  train.zip
  inflating: ./train/Train.csv       


In [7]:
fname_train="./train/Train.csv"
df=pd.read_csv(fname_train)

In [8]:
df.head(5)

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,ID_3SSHI56C,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,...,1.469838,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,ok,ok,ok
1,ID_599OOLZA,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,...,1.552979,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,ok,high,high
2,ID_MVJGPQ75,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,...,1.516723,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,ok,ok,high
3,ID_CK6RF8YV,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,...,1.442957,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,low,ok,high
4,ID_82N6QE6I,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.55

In [9]:
df.columns

Index(['Reading_ID', 'absorbance0', 'absorbance1', 'absorbance2',
       'absorbance3', 'absorbance4', 'absorbance5', 'absorbance6',
       'absorbance7', 'absorbance8',
       ...
       'absorbance165', 'absorbance166', 'absorbance167', 'absorbance168',
       'absorbance169', 'temperature', 'humidity', 'hdl_cholesterol_human',
       'hemoglobin(hgb)_human', 'cholesterol_ldl_human'],
      dtype='object', length=176)

In [10]:
NUM_OF_COMP=3

In [11]:
PCA_X=df.iloc[:,1:-3]

In [15]:
import numpy as np
from sklearn.decomposition import PCA

O_pca = PCA(n_components=NUM_OF_COMP)
O_PCA_values=O_pca.fit_transform(PCA_X,)

print(O_pca.explained_variance_ratio_)

print(O_pca.singular_values_)

[0.88359254 0.10652794 0.00689595]
[935.94477398 324.97950563  82.68396585]


In [16]:
import plotly.express as px

fig = px.scatter(df, x="absorbance82", y="absorbance140",color="hemoglobin(hgb)_human")
fig.show()

In [17]:
import plotly.express as px
opc_df=pd.DataFrame()
opc_df["PCA_x"]=O_PCA_values[:,0]
opc_df["PCA_y"]=O_PCA_values[:,1]
opc_df["color"]=df["hemoglobin(hgb)_human"]


fig = px.scatter(opc_df,x="PCA_x", y="PCA_y",color="color")
fig.show()

In [62]:
import plotly.express as px
opc_df=pd.DataFrame()
opc_df["PCA_x"]=O_PCA_values[:,0]
opc_df["PCA_y"]=O_PCA_values[:,1]
opc_df["color"]=df["hemoglobin(hgb)_human"]
opc_df["labels"]=labels


fig = px.scatter(opc_df,x="PCA_x", y="PCA_y",color="labels")
fig.show()

In [123]:
def label_2_num(label):
    out=[]
    for l in label:
        
        if l=="low":
            o=1
        if l=="ok":
            o=2
        if l=="high":
            o=3
        out.append(o)
        
    return(out)

from sklearn.cluster import DBSCAN
# Compute DBSCAN
dbscan_inp=pd.DataFrame()
dbscan_inp["temperature"]=df["temperature"]
dbscan_inp["humidity"]=df["humidity"]
dbscan_inp["cholesterol_ldl_human"]=label_2_num(list(df["cholesterol_ldl_human"]))
dbscan_inp["hemoglobin(hgb)_human"]=label_2_num(list(df["hemoglobin(hgb)_human"]))




db = DBSCAN(eps=0.85, min_samples=20,).fit(dbscan_inp)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
dbscan_inp["labels"]=labels


fig = px.scatter(dbscan_inp,x="temperature", y="humidity",color="labels",opacity=0.3)
fig2 = px.scatter(dbscan_inp,x="temperature", y="humidity",color="hemoglobin(hgb)_human",opacity=0.3)
fig.show()
fig2.show()


In [105]:
list(df["hemoglobin(hgb)_human"])

['ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'low',
 'ok',
 'low',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'low',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'low',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'high',
 'high',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'high',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'low',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok'

In [111]:
def one_hot_encode_df(df,str_index):
    from sklearn.preprocessing import OneHotEncoder
    indf=df.copy(True)
    one_hot=OneHotEncoder()
    col=indf[str_index].values
    col=col.reshape(-1,1)
    one_hot.fit(col)
    out=one_hot.transform(col).toarray()
    codes=one_hot.get_feature_names()
    print(codes)
    for ind,code in enumerate(codes):
        print("Itt:",ind,codes[ind])
        indf[str_index+"_"+code]=out[:,ind]
    return(indf,one_hot)


In [112]:
hdl_col_df,hdl_onehot=one_hot_encode_df(df,"hdl_cholesterol_human")
hemo_df,hemo_onehot=one_hot_encode_df(df,"hemoglobin(hgb)_human")
col_df,col_onehot=one_hot_encode_df(df,"cholesterol_ldl_human")

['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [113]:
hemo_df.tail()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok
13135,ID_NGPC0DA3,0.483513,0.482732,0.487531,0.497946,0.500697,0.505740,0.511904,0.519018,0.529455,0.540356,0.554324,0.563337,0.568932,0.573057,0.575291,0.574014,0.573149,0.570979,0.567632,0.564020,0.560878,0.557749,0.554752,0.552457,0.548313,0.547369,0.544267,0.542225,0.542338,0.539435,0.539836,0.539883,0.540450,0.541485,0.542836,0.544809,0.548269,0.552457,0.556129,...,1.372840,1.357367,1.342472,1.334482,1.326614,1.314757,1.301887,1.292316,1.296431,1.281496,1.278615,1.268085,1.256004,1.255005,1.245078,1.245569,1.231796,1.228750,1.238677,1.230577,1.218419,1.224957,1.220593,1.218087,1.214382,1.208455,1.202338,1.189651,1.182641,1.206917,1.188173,1.188303,34.79,36.93,high,ok,ok,0.0,0.0,1.0
13136,ID_XRBUD5U8,0.525435,0.527563,0.528863,0.531776,0.541156,0.547318,0.552549,0.559060,0.566892,0.581571,0.595248,0.603612,0.611313,0.614295,0.617476,0.617077,0.618908,0.616631,0.610841,0.604681,0.601588,0.598501,0.594149,0.589455,0.587673,0.584746,0.581093,0.580218,0.578071,0.577839,0.576454,0.576026,0.576220,0.579007,0.581564,0.584409,0.587786,0.591168,0.595874,...,1.545593,1.521200,1.518922,1.508497,1.501945,1.487737,1.481170,1.457265,1.444710,1.452271,1.428153,1.421319,1.419508,1.411549,1.415561,1.405296,1.390187,1.384455,1.383000,1.380202,1.383244,1.378410,1.377784,1.359367,1.379885,1.378378,1.367057,1.388165,1.375223,1.372435,1.395317,1.387709,43.12,19.14,ok,ok,ok,0.0,0.0,1.0
13137,ID_2M9L5NV2,0.512718,0.517815,0.524857,0.525466,0.536542,0.542930,0.550628,0.558939,0.567593,0.579192,0.595752,0.609749,0.617717,0.621270,0.625333,0.626705,0.623455,0.622948,0.618986,0.616963,0.614392,0.610574,0.606732,0.604206,0.601527,0.597425,0.595800,0.592601,0.590437,0.589152,0.588726,0.587134,0.588214,0.589699,0.590376,0.592544,0.594104,0.599264,0.603341,...,1.509437,1.497262,1.487541,1.481796,1.460926,1.458425,1.445150,1.430362,1.427516,1.420605,1.414007,1.415796,1.395715,1.399176,1.392111,1.384322,1.376676,1.364141,1.363428,1.366804,1.371499,1.367140,1.364490,1.367668,1.362808,1.385546,1.386140,1.396862,1.377644,1.428150,1.447840,1.384221,42.48,43.41,ok,ok,ok,0.0,0.0,1.0
13138,ID_C5V5SD2D,0.456747,0.472575,0.466935,0.466698,0.478546,0.486451,0.494838,0.496540,0.508274,0.521304,0.532652,0.542257,0.550854,0.556305,0.558084,0.559439,0.558418,0.554719,0.554852,0.552821,0.551641,0.549722,0.545233,0.542932,0.539894,0.536520,0.534783,0.530619,0.531076,0.530187,0.528713,0.527754,0.529230,0.529761,0.531586,0.533869,0.535897,0.539685,0.543911,...,1.350871,1.331415,1.326455,1.322705,1.310202,1.302262,1.294962,1.287724,1.283147,1.269678,1.257537,1.259994,1.248000,1.253335,1.237789,1.228726,1.224605,1.224872,1.217346,1.225207,1.224785,1.214651,1.214882,1.214873,1.217364,1.208545,1.185192,1.181483,1.172430,1.174544,1.154095,1.212907,41.86,35.10,ok,ok,ok,0.0,0.0,1.0
13139,ID_9R5ZR9NX,0.498372,0.501692,0.505924,0.510975,0.516707,0.520821,0.531242,0.5373

In [114]:
del hdl_col_df["hdl_cholesterol_human"]
del hdl_col_df["hemoglobin(hgb)_human"]
del hdl_col_df["cholesterol_ldl_human"]
del hemo_df["hdl_cholesterol_human"]
del hemo_df["hemoglobin(hgb)_human"]
del hemo_df["cholesterol_ldl_human"]
del col_df["hdl_cholesterol_human"]
del col_df["hemoglobin(hgb)_human"]
del col_df["cholesterol_ldl_human"]




In [115]:
hdl_col_df.head()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok
0,ID_3SSHI56C,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,...,1.469838,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,0.0,0.0,1.0
1,ID_599OOLZA,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,...,1.552979,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,0.0,0.0,1.0
2,ID_MVJGPQ75,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,...,1.516723,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,0.0,0.0,1.0
3,ID_CK6RF8YV,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,...,1.442957,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,0.0,1.0,0.0
4,ID_82N6QE6I,0.510485,0.519359,0.524225,0.528419,0.53527

In [116]:
hdl_col_df.describe()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok
count,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,...,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000
mean,0.507347,0.509800,0.513687,0.519391,0.525976,0.533482,0.539453,0.544755,0.552902,0.565279,0.578415,0.590215,0.597083,0.600618,0.602828,0.603434,0.602075,0.599335,0.596222,0.593066,0.589816,0.586821,0.583665,0.580336,0.577325,0.574321,0.571803,0.569906,0.568239,0.566951,0.566204,0.565929,0.566327,0.567358,0.568966,0.571237,0.574091,0.577933,0.581767,0.585947,...,1.494436,1.481857,1.470147,1.458943,1.447029,1.436010,1.424403,1.413153,1.402854,1.394788,1.382297,1.373392,1.364888,1.356420,1.348587,1.341277,1.333918,1.327140,1.320583,1.314208,1.309417,1.305020,1.301597,1.299224,1.298162,1.297279,1.296103,1.292939,1.287715,1.279190,1.271131,1.264782,1.262234,1.274563,1.261993,41.088709,31.534744,0.196347,0.232877,0.570776
std,0.042075,0.041587,0.041525,0.041676,0.041125,0.040644,0.040715,0.040946,0.040820,0.041038,0.041164,0.041268,0.041501,0.041560,0.041508,0.041379,0.041080,0.040716,0.040453,0.040368,0.040127,0.040101,0.040019,0.039927,0.039915,0.039767,0.039740,0.039792,0.039787,0.039715,0.039708,0.039738,0.039781,0.039860,0.039862,0.039979,0.040065,0.040266,0.040384,0.040327,...,0.073972,0.072704,0.071497,0.070351,0.069132,0.068116,0.066951,0.066179,0.065450,0.065161,0.063811,0.063188,0.062849,0.062148,0.061847,0.061448,0.060891,0.060457,0.060267,0.059903,0.059883,0.059731,0.059607,0.059692,0.060137,0.060583,0.062033,0.064989,0.069659,0.077427,0.085519,0.092525,0.097079,0.103298,0.102908,2.964205,8.118994,0.397249,0.422680,0.494984
min,0.237409,0.251987,0.269550,0.284636,0.302883,0.318334,0.333950,0.345124,0.361352,0.374106,0.388672,0.407299,0.416705,0.420315,0.423078,0.423848,0.423839,0.422310,0.421942,0.419578,0.416689,0.414667,0.413794,0.410312,0.408713,0.405934,0.403518,0.402689,0.401398,0.400746,0.399525,0.399710,0.399237,0.400620,0.403256,0.404547,0.407792,0.412233,0.416269,0.421089,...,1.258771,1

In [117]:
minmax=MinMaxScaler()
minmax.fit(hdl_col_df.iloc[:,1:])
data_transform=minmax.transform(hdl_col_df.iloc[:,1:])

In [118]:
X=data_transform[:,:-3]
y_hdl_col=hdl_col_df.iloc[:,-3:]
y_hemo=hemo_df.iloc[:,-3:]
y_hemo=hemo_df.iloc[:,-3:]
y_col=col_df.iloc[:,-3:]

In [119]:
import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences

In [220]:
def blood_model():
    #clear_session()



    input_len=len(X[0])

    print(input_len)
    output_size=3
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,),)

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="normal"
    d1=Dense(7,activation="selu",kernel_initializer=initializer,input_dim=input_len,name="bottle")(input1)
    d1=Dropout(drop_frac0)(d1)
    


    pred=Dense(input_len,activation="selu")(d1)

    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adamax(learning_rate=0.01)


    #lossfn = tf.keras.metrics.MSE()

    model.compile(loss="MSE",
        optimizer=opt,
        metrics=["MSE"])
    return(model)

In [221]:
!rm *.hdf5

In [222]:
model_name="encoder"
bestmodelname=""
def scheduler(epoch, lr):
    return 0.001

callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

def save_best_model_name():
    bestmodelname=model_name+"f_{val_MSE:.5f}_.hdf5"

callbacks =[
            callback_LR,
        
            ModelCheckpoint(filepath=model_name+"_{val_MSE:.5f}_.hdf5", monitor='val_MSE',
                        verbose=1, save_best_only=True, mode='min'),
           
        ]

In [223]:
full_model=blood_model()

172


In [224]:
save_best_model_name()

In [225]:
bestmodelname

''

In [227]:
full_model.fit(X,
          X,
          epochs=450, 
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=callbacks,
          shuffle=True
          
          )

Epoch 1/450
318/329 [===========================>..] - ETA: 0s - loss: 2.1004e-04 - MSE: 2.1004e-04
Epoch 00001: val_MSE did not improve from 0.00021
329/329 [==============================] - 1s 5ms/step - loss: 2.1004e-04 - MSE: 2.1004e-04 - val_loss: 2.0979e-04 - val_MSE: 2.0979e-04 - lr: 0.0010
Epoch 2/450
325/329 [============================>.] - ETA: 0s - loss: 2.0956e-04 - MSE: 2.0956e-04
Epoch 00002: val_MSE did not improve from 0.00021
329/329 [==============================] - 2s 5ms/step - loss: 2.0967e-04 - MSE: 2.0967e-04 - val_loss: 2.1018e-04 - val_MSE: 2.1018e-04 - lr: 0.0010
Epoch 3/450
329/329 [==============================] - ETA: 0s - loss: 2.0951e-04 - MSE: 2.0951e-04
Epoch 00003: val_MSE did not improve from 0.00021
329/329 [==============================] - 2s 5ms/step - loss: 2.0951e-04 - MSE: 2.0951e-04 - val_loss: 2.0980e-04 - val_MSE: 2.0980e-04 - lr: 0.0010
Epoch 4/450
318/329 [===========================>..] - ETA: 0s - loss: 2.0942e-04 - MSE: 2.0942e-04


In [228]:
X[0]

array([0.57629797, 0.56316263, 0.56058608, 0.55348446, 0.55432924,
       0.52801177, 0.52429853, 0.50529651, 0.49396945, 0.4908818 ,
       0.47677241, 0.47466856, 0.46398309, 0.46296189, 0.45314056,
       0.4461359 , 0.44496283, 0.44416027, 0.43586569, 0.43853278,
       0.43820073, 0.43619613, 0.43587531, 0.43594288, 0.42574665,
       0.43109712, 0.43362324, 0.43267825, 0.42976148, 0.43394287,
       0.43184588, 0.43866202, 0.4327672 , 0.431208  , 0.43553574,
       0.4331199 , 0.43502257, 0.4306428 , 0.43040646, 0.42566461,
       0.42792858, 0.42574241, 0.42172644, 0.42014705, 0.41649629,
       0.40968119, 0.4075168 , 0.40835163, 0.40112821, 0.39484841,
       0.39387877, 0.38614882, 0.38647705, 0.38217006, 0.37493735,
       0.3699833 , 0.36920971, 0.36554815, 0.36796923, 0.37310292,
       0.36670567, 0.36726969, 0.36314847, 0.36048029, 0.35013203,
       0.34812045, 0.34701882, 0.34821374, 0.34586192, 0.34227702,
       0.33816833, 0.33752315, 0.3293699 , 0.32742852, 0.32797

In [229]:
full_model_bottle=Model(full_model.inputs,full_model.get_layer("bottle").output)




In [230]:
full_pred_bottle=full_model_bottle.predict(X)



In [231]:
full_pred_bottle

array([[ 0.73612803, -0.18728596,  1.1215092 , ..., -0.2059155 ,
         0.78768754,  0.5842162 ],
       [ 0.71795666, -0.24121678,  1.7137302 , ..., -0.13969636,
         0.9261125 ,  0.7858857 ],
       [ 0.77321607, -0.27385622,  1.5683258 , ..., -0.22148277,
         0.4820413 ,  0.4825989 ],
       ...,
       [ 1.2888013 , -0.250663  ,  1.887331  , ...,  0.01227529,
         0.5899405 ,  0.88422656],
       [ 0.9488216 , -0.16394228,  0.6836511 , ..., -0.16270256,
         0.12849388,  0.36808652],
       [ 0.97089726, -0.30241537,  2.0494888 , ..., -0.18110922,
         0.69242835,  0.84945154]], dtype=float32)

In [232]:
df_bottle=pd.DataFrame()
df_bottle["summary"]=df["hdl_cholesterol_human"]+"+"+df["hemoglobin(hgb)_human"]+"+"+df["cholesterol_ldl_human"]


In [233]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [234]:
# def a search matrix
def create_search_list(pred,df,str_index):
    out=[]
    for i,vec in enumerate(pred):
        out.append([vec,df[str_index].values[i]])
    return(out)
    
    

In [235]:
def generate_similarity(it1,sim_table):
    out=[]
    for it2 in sim_table:
        
        diff=cosine_similarity(it1,it2[0])
        if 1-diff<0.006:
            
            out.append(it2[1])
    return(out)

In [236]:
def find_max(dict):
    max=0
    sum=0
    outkey=""
    for key in dict.keys():
        akt=dict[key]
        sum+=akt
        if max<akt:
            max=akt
            outkey=key
    return outkey,max

In [237]:
def statistic(a):
    c=list(set(a))
    #print(c)
    out={}
    for i in c:
        out[i]=a.count(i)
    return(out)

In [238]:
full_bottle_list=create_search_list(full_pred_bottle,df_bottle,"summary")

In [239]:
import numpy as np




In [240]:
def calc_status(pred,dfcolumn):
    i=0
    for index,element in enumerate(pred):

        if element != dfcolumn.iloc[index]:
            #print(f"{index}")
            i +=1
    print(f"{i} hibás, {i/len(pred)*100} % hiba")        

In [241]:
# submission generation
!rm "./Test.csv"

In [242]:
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv

--2021-12-05 17:08:31--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv [following]
--2021-12-05 17:08:31--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10299206 (9.8M) [text/plain]
Saving to: ‘Test.csv’

Test.csv            100%[===================>]   9.82M  --.-KB/s    in 0.1s    

2021-12-05 17:08:32 (74.2 MB/s) - ‘Test.csv’ saved

In [243]:
test_df=pd.read_csv("./Test.csv")
test_df.head()

test_df['hdl_cholesterol_human']=0
test_df["hemoglobin(hgb)_human"]=0
test_df["cholesterol_ldl_human"]=0

In [244]:
test_transform=minmax.transform(test_df.iloc[:,1:])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cholesterol_ldl_human
- hdl_cholesterol_human
- hemoglobin(hgb)_human
Feature names seen at fit time, yet now missing:
- hdl_cholesterol_human_x0_high
- hdl_cholesterol_human_x0_low
- hdl_cholesterol_human_x0_ok

  warnings.warn(message, FutureWarning)


In [245]:
X_test=test_transform[:,:-3]

In [246]:
# load models
from keras.models import load_model
base_path="./"

bottle_name=base_path+"encoder_0.00020_.hdf5"

bottle_model=load_model(bottle_name)


In [247]:
bottle_pred_test=full_model_bottle.predict(X_test)


In [248]:
bottle_pred_test[0]

array([ 1.0610826 , -0.09461067,  0.9018101 , -0.03815306, -0.2768333 ,
       -0.17918582,  0.4432311 ], dtype=float32)

In [249]:
full_predicted_test=[]


In [250]:
test_result=[]
db=0
for test_i in bottle_pred_test:
    o1=generate_similarity( test_i, full_bottle_list)
    result,_=find_max(statistic(o1))
    if result=="":
        result="NaN"
    db+=1
    if (db % 100)==0:
        print("...")
    print(result,end=",")
    test_result.append(result)

NaN,ok+ok+ok,NaN,NaN,ok+ok+ok,low+ok+high,ok+ok+ok,NaN,high+ok+high,low+ok+ok,NaN,high+ok+ok,low+ok+low,NaN,ok+ok+ok,ok+ok+ok,NaN,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+high,NaN,NaN,ok+ok+ok,ok+ok+high,ok+ok+ok,ok+ok+ok,ok+ok+ok,low+ok+ok,low+ok+ok,ok+ok+high,ok+ok+ok,NaN,ok+ok+ok,ok+ok+ok,NaN,NaN,ok+ok+ok,ok+ok+ok,low+low+ok,ok+ok+ok,low+ok+ok,NaN,low+low+high,ok+ok+high,low+ok+ok,ok+ok+ok,ok+ok+high,low+low+high,ok+low+high,high+ok+ok,ok+ok+ok,high+ok+high,ok+ok+ok,ok+ok+high,low+ok+ok,ok+ok+ok,ok+ok+high,high+ok+ok,ok+ok+ok,high+ok+ok,ok+ok+high,ok+ok+ok,low+ok+low,ok+ok+high,low+ok+ok,ok+ok+ok,NaN,low+ok+ok,ok+ok+ok,NaN,ok+ok+high,ok+low+ok,low+ok+low,high+ok+ok,ok+ok+high,ok+ok+high,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+ok,low+high+low,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+high,ok+ok+high,low+ok+ok,ok+ok+ok,ok+ok+ok,ok+ok+ok,ok+low+high,low+ok+ok,ok+ok+high,ok+ok+ok,ok+ok+ok,...
high+ok+high,ok+ok+ok,ok+ok+ok,low+ok+ok,high+ok+ok,NaN,NaN,ok+ok+ok,ok+ok+ok,NaN,ok+ok+high

In [251]:
def de_conv(pred):
    out0=[]
    out1=[]
    out2=[]

    for p in pred:
        if p=="NaN":
            p="ok+ok+ok"
        
        out0.append(p.split("+")[0])
        out1.append(p.split("+")[1])
        out2.append(p.split("+")[2])
        
    return(out0,out1,out2)

In [252]:
ret0,ret1,ret2=de_conv(test_result)

In [253]:
outdf=pd.DataFrame()
outdf["Reading_ID"]=test_df["Reading_ID"]
outdf["hdl_cholesterol_human"]=ret0
outdf["hemoglobin(hgb)_human"]=ret1
outdf["cholesterol_ldl_human"]=ret2

outdf.set_index("Reading_ID",inplace=True)

In [254]:
outdf.head(1)

,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
Reading_ID,,,
ID_37BEI22R,ok,ok,ok


In [255]:
!rm SampleSubmission.csv
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv

--2021-12-05 17:17:25--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv [following]
--2021-12-05 17:17:25--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 384318 (375K) [text/plain]
Saving to: ‘SampleSubmission.csv’

SampleSubmission.cs 100%[===================>] 375.31K  --.-KB/s    in 0.03s   

2021

In [256]:
submission_df=pd.read_csv("./SampleSubmission.csv")

In [257]:
for i in submission_df.index:
    #print(i)
    indexer=submission_df.loc[i,"Reading_ID"]

    splitter=indexer.split("_",2)
    out_index=splitter[0]+"_"+splitter[1]
    col_index=splitter[2]
    #print(out_index)
    result=outdf.loc[out_index][col_index]
    submission_df.iloc[i,1]=result

In [258]:
submission_df.set_index("Reading_ID", inplace=True)

In [259]:
def create_submission_name(project):
    from datetime  import datetime
    strdate=datetime.now().strftime("%Y%m%d_%H%M%S")
    out=project+"_"+"submission"+"_"+strdate+".csv"
    return(out)

In [260]:
submission_fname=create_submission_name("BloodAI")

In [261]:
submission_fname

'BloodAI_submission_20211205_171730.csv'

In [262]:
submission_df.to_csv(submission_fname,index=True)